In [2]:
import pandas as pd
academic = pd.read_feather('./feather/academic.feather')#.to_csv('./tables/academics.csv',index=False)

# Tables concerning inproceedings (conference articles)
improceeding = pd.read_feather('./feather/improceeding_table.feather')#.to_csv('./tables/improceedings=.csv',index=False)
author_inproceeding = pd.read_feather('./feather/author_inproceedings.feather')#.to_csv('./tables/inproceedings_authors.csv',index=False)
booktitle_improceeding = pd.read_feather('./feather/improceeding_booktitle.feather')#.to_csv('./tables/improceedings_booktitles.csv',index=False)

# All tables concerning articles(journal articles)
article = pd.read_feather('./feather/articles_table.feather')#.to_csv('./tables/articles.csv',index=False)
author_article = pd.read_feather('./feather/author_article.feather')#.to_csv('./tables/article_authors.csv',index=False)
journal = pd.read_feather('./feather/journals_table.feather')#.to_csv('./tables/journals.csv',index=False)


# All tables concerning proceedings (conference proceedings => bundled conference articles)
editor_proceeding = pd.read_feather('./feather/editor_proceedings.feather')#.to_csv('./tables/proceedings_editors.csv',index=False)
publisher_proceeding = pd.read_feather('./feather/publisher_proceeding.feather')#.to_csv('./tables/proceedings_publishers.csv',index=False)
publisher = pd.read_feather('./feather/publisher_table.feather')#.to_csv('./tables/publishers.csv',index=False)
proceeding = pd.read_feather('./feather/proceeding_table.feather')#.to_csv('./tables/proceedings.csv',index=False)
booktitle_proceeding = pd.read_feather('./feather/proceeding_booktitle.feather')#.to_csv('./tables/proceedings_booktitles.csv',index=False)

# All tables concerning booktiles (conferences)
booktitle = pd.read_feather('./feather/booktitles_table.feather')#.to_csv('./tables/booktitles.csv',index=False)

In [3]:
article

,key,title,journal_id
0,conf/www/RamamohanaraoF07,Patterns Based Classifiers.,1936
1,conf/www/ZhaiL07,Extracting Web Data Using Instance-Based Learn...,1936
2,conf/www/NguKN07,Editorial.,1936
3,conf/www/Ericsson07,The Effects of XML Compression on SOAP Perform...,1936
4,conf/www/MartinBMMPSMSS07,Bringing Semantics to Web Services with OWL-S.,1936
...,...,...,...
2987941,tr/gte/TR-0244-12-93-165,MetaObject Protocol Concepts for a RISC Object...,545
2987942,tr/gte/TR-0146-06-91-165,Towards a Transaction Management System for DOM.,545
2987943,tr/gte/TR-0222-10-92-165,DARWIN: On the Incremental Migration of Legacy...,545
2987944,tr/dec/SRC1997-018,"The 1995 SQL Reunion: People, Project, and Pol...",395


In [4]:
#rename columns of tables
academic.columns = ['academic_id','academic_name']
article.columns = ['article_id','article_title','journal_id']
author_article.columns = ['academic_id','article_id']
journal.columns = ['journal_id','journal_name']
proceeding.columns = ['proceeding_id','proceeding_title','year','volume']
publisher.columns = ['publisher_id','publisher_name']
booktitle.columns = ['booktitle_id','booktitle_name']
booktitle_proceeding.columns = ['proceeding_id','booktitle_id']
improceeding.columns = ['improceeding_id','improceeding_title','pages','year']
booktitle_improceeding.columns = ['improceeding_id','booktitle_id']
publisher_proceeding.columns = ['publisher_id','proceeding_id']
editor_proceeding.columns = ['academic_id','proceeding_id']


In [7]:
import duckdb
queryE1 = "Select publisher.publisher_name, proceeding.proceeding_title FROM publisher JOIN publisher_proceeding ON publisher.publisher_id=publisher_proceeding.publisher_id JOIN proceeding ON proceeding.proceeding_id=publisher_proceeding.proceeding_id WHERE proceeding.proceeding_title LIKE '%PODS%'"
duckdb.query(queryE1).to_df()
#publisher.value_counts()

,publisher_name,proceeding_title
0,NaN,"The ACM SIGMOD Anthology, CDROM Edition, Volum..."
1,ACM,Proceedings of the ACM SIGMOD/PODS PhD Symposi...
2,ACM,Proceedings of the ACM SIGMOD/PODS PhD Symposi...
3,ACM,Proceedings of the ACM SIGMOD/PODS PhD Symposi...
4,ACM,Proceedings of the ACM SIGMOD/PODS PhD Symposi...
...,...,...
282369,ACM,Proceedings of the Twenty-Ninth ACM SIGMOD-SIG...
282370,ACM,Proceedings of the Twenty-Ninth ACM SIGMOD-SIG...
282371,ACM,Proceedings of the Twenty-Ninth ACM SIGMOD-SIG...
282372,ACM,Proceedings of the Twenty-Ninth ACM SIGMOD-SIG...


In [ ]:
queryE2_1 = "SELECT article.article_title, journal.journal_name, academic.academic_name FROM article JOIN author_article ON author_article.article_id=article.article_id JOIN academic ON academic.academic_id=author_article.academic_id JOIN journal ON article.journal_id=journal.journal_id WHERE journal.journal_name='Theory Comput. Syst.' AND academic.academic_name='Martin Grohe' ORDER BY article.article_title"
queryE2_2 = "SELECT article.article_title, journal.journal_name, academic.academic_name FROM article, journal, author_article, academic WHERE article.journal_id=journal.journal_id AND article.article_id=author_article.article_id AND academic.academic_name='Martin Grohe' AND journal_name='Theory Comput. Syst.'"
duckdb.query(query2_1).to_df()

,article_title,journal_name,academic_name
0,Database Query Processing Using Finite Cursor ...,Theory Comput. Syst.,Martin Grohe
1,Learnability and Definability in Trees and Sim...,Theory Comput. Syst.,Martin Grohe
2,Tight Lower and Upper Bounds for the Complexit...,Theory Comput. Syst.,Martin Grohe


In [21]:
queryM1 = "Select * FROM proceeding"
duckdb.query(queryM1).to_df()

,proceeding_id,proceeding_title,year,volume
0,reference/med/2013,Handbook of Medical and Healthcare Technologies,2013,NaN
1,conf/coopis/2003,On The Move to Meaningful Internet Systems 200...,2003,2888
2,conf/coopis/2004-2,On the Move to Meaningful Internet Systems 200...,2004,3291
3,conf/coopis/2004-2,On the Move to Meaningful Internet Systems 200...,2004,3291
4,conf/coopis/2004-2,On the Move to Meaningful Internet Systems 200...,2004,3291
...,...,...,...,...
53398,series/ihis/2009hoo,Handbook on Ontologies,2009,NaN
53399,series/ihis/2015bpm1,"Handbook on Business Process Management 1, Int...",2015,NaN
53400,series/ihis/2015bpm2,"Handbook on Business Process Management 2, Str...",2015,NaN
53401,series/sapere/2013-5,Philosophy and Theory of Artificial Intelligen...,2013,5


In [22]:
queryM2 = "Select * FROM proceeding ORDER BY proceeding.year"
duckdb.query(queryM2).to_df()

,proceeding_id,proceeding_title,year,volume
0,conf/aieeire/1951,Papers and discussions presented at the 1951 J...,1951,NaN
1,conf/acm/1952t,Proceedings of the 1952 ACM national meeting (...,1952,NaN
2,conf/acm/1952p,Proceedings of the 1952 ACM national meeting (...,1952,NaN
3,conf/aieeire/1953e,Papers and discussions presented at the 1953 e...,1953,NaN
4,conf/aieeire/1953w,Proceedings of the 1953 western computer confe...,1953,NaN
...,...,...,...,...
53398,conf/intellisys/2022-1,Intelligent Systems and Applications - Proceed...,2023,542
53399,conf/intellisys/2022-2,Intelligent Systems and Applications - Proceed...,2023,543
53400,conf/intellisys/2022-3,Intelligent Systems and Applications - Proceed...,2023,544
53401,conf/dps/2022,Intelligent and Safe Computer Systems in Contr...,2023,545


In [ ]:
queryM3 = ""
duckdb.query(queryM3).to_df()

In [23]:
queryM4 = ""
duckdb.query(queryM4).to_df()

In [21]:
queryM5 = "SELECT COUNT(academic.academic_name) AS editor_frequency, academic.academic_name FROM academic JOIN editor_proceeding ON academic.academic_id=editor_proceeding.academic_id JOIN proceeding ON proceeding.proceeding_id=editor_proceeding.proceeding_id WHERE proceeding.proceeding_title LIKE '%PODS%' GROUP BY academic.academic_name ORDER BY editor_frequency DESC"
duckdb.query(queryM5).to_df()

,editor_frequency,academic_name
0,4,Xin Luna Dong
1,4,Peter A. Boncz
2,3,Maurizio Lenzerini
3,3,Dan Suciu
4,3,Josep Lluís Larriba-Pey
5,2,M. Tamer Özsu
6,2,Leonid Libkin
7,2,Richard Hull 0001
8,2,Jan Paredaens
9,2,Tova Milo


In [24]:
queryM6 = ""
duckdb.query(queryM6).to_df()

In [ ]:
queryH1 = ""
duckdb.query(queryH1).to_df()

In [25]:
queryH2 = ""
duckdb.query(queryH2).to_df()

In [ ]:
queryB1 = ""
duckdb.query(queryB1).to_df()

In [ ]:
queryB2 = ""
duckdb.query(queryB2).to_df()